In [ ]:
head = 'HEAD PRINT FORMAT   0\n'\
'HEAD SAVE UNIT   336\n'\
'DRAWDOWN PRINT FORMAT   0\n\n'
head

In [ ]:
kper = range(1,1100)
step = range(1,25)

with open('RR_2022/rr_all.oc','w') as wr:
    wr.write(head)
    for k in kper:
        
        for s in step:
            wr.write(f"period {k} step {s}\n")
            # wr.write("PRINT BUDGET\n")
            # wr.write("SAVE BUDGET\n")
            
            if (s==24):
                wr.write("PRINT BUDGET\n")
                wr.write('SAVE HEAD\n\n')
            
            # wr.write('\n\n')


In [ ]:
260+109

In [ ]:
for i in range(260):
    print('-1')

In [ ]:
import basic
import postprocess

In [ ]:
ml = basic.load_model(path = 'temp/June2013')
start_date = '1/1/2013'

In [ ]:
ISWRPQAQ , ISWRPRGF, ISWRPSTG, ISWRPSTR, ISWRPQM = postprocess.SWR(ml, start_date, max_reach = 116, remove_ponds = True  )

In [ ]:
ISWRPQAQ.loc[:, 'STAGE'].droplevel([1, 2, 3, 4, 6]).groupby(level = [0,1]).mean().unstack()

In [ ]:
ISWRPQAQ.loc[:, 'STAGE'].droplevel([1, 2, 3, 4, 6]).unstack()

In [ ]:
import run_parallel


In [ ]:
run_parallel.par_run('June2019',check_for_success=False, skip_setup=  True, 
                     copyfiles_from_base = False, post_process_only = True)

In [ ]:
import hydro_context
from datetime import datetime
import conda_scripts.arich_functions as af
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
rr, dry, mw, total = hydro_context.get_data()

In [ ]:
def julian_water_year(wy):
    '''
    return days from start of water year, creates pseudo date from start of WY 2020.
    Args:
        date: SERIES of dates (does not work with datetime index)

    Returns:

    '''
    if isinstance(wy, pd.DatetimeIndex):
        wy = pd.DataFrame(wy, columns =[ 'Date'])
    elif isinstance(wy, pd.Series):
        wy = wy.to_frame('Date')
    else:
        raise AssertionError(f"type is not supported {type(wy)}")
        
    wy.loc[:,'month'] = wy.Date.dt.month
    wy.loc[:,'year'] = wy.Date.dt.year
    wy.loc[:,'day'] = wy.Date.dt.day
    
    c  = wy.loc[:,'month']<10
    wy.loc[:,'year'] = 2000
    wy.loc[~c,'year'] = 1999
    wy.loc[:, 'WY_date'] = pd.to_datetime(wy.loc[:,['year','month','day']])
    
    wy = wy.loc[:,'WY_date'].values
    
    return wy

In [ ]:
def plot_by_wy(df1,df2, df3, df4, year = 2018):
    fig, ax = plt.subplots(4,1, sharex = True, figsize = (10,6), )
    
    
    for cnt, df in enumerate([df1, df2, df3, df4]):
        for group, df  in get_wy(df).groupby('Water Year'):
            df = df.sort_index()
            if group == year:
                c = 'k'
                lw = 3
            else:
                c = None
                lw = 1
            ax[cnt].plot(df.loc[:,'Date'], df.iloc[:,0], label =f"WY {group}", c =c, lw = lw)

        ax[cnt].set_yscale("log")
        ax[cnt].set_title(df.columns[0])
        ax[cnt].set_ylabel('cfs')
        ax[cnt].grid(True)
    ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))
    
    ax[0].xaxis.set_minor_locator(mdates.MonthLocator(range(1,13)))
    ax[0].xaxis.set_major_locator(mdates.MonthLocator(range(1,13)))
    ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    
def get_wy(ser):
    # ser.loc[:,'Water Year'] = af.water_year(ser)
    ser.loc[:, 'Water Year'] = af.water_year(ser.index)
    ser.loc[:,'Date'] = julian_water_year(ser.index.to_series())
    
    
    return ser

In [ ]:
plot_by_wy(rr, dry, mw, total)

In [ ]:
julian_water_year(rr.index)

In [ ]:
get_wy(rr)

In [ ]:
fig, ax = plt.subplots()
for group,df  in get_wy(rr).groupby('Water Year'):
    df = df.sort_index()
    ax.plot(df.loc[:,'Date'], df.iloc[:,0], label = group)
ax.legend()
ax.set_yscale('log')

In [ ]:
df

In [ ]:
df.set_index('Date').drop(columns = 'Water Year').plot()

In [ ]:
df.set_index('Date').drop(columns = 'Water Year').sort_index()

In [ ]:
df.sort_index()